# TRYING/TESTING necessary libraries for document processing #


## STEP 1: Reading and extracting texts from various document formats ##

In [5]:
from pathlib import Path
file_path = Path("data/raw_documents/receipts/train_ticket_receipt.pdf").resolve()
print("Resolved Path:", file_path)


Resolved Path: C:\Users\User\Documents\SideProject\personal_doc_chatbot_with_RAG\data\raw_documents\receipts\train_ticket_receipt.pdf


#### Extract from PDFs ####


In [25]:
# Reading and extracting texts from PDFs
# by using PyMuPDF
import pymupdf4llm

#use ticket_train receipt as an example
md_text = pymupdf4llm.to_markdown("c:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/personal_documents/current_resume.pdf")

print(md_text)




Processing c:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/personal_documents/current_resume.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
# t s ters

Current CGPA 3.49

## PROFESSIONAL SUMMARY

A final year student of Bachelor in Computer Science with the goal to seek and expand knowledge and
specialize in Software Engineering field. With a strong passion for software engineering, I am actively
seeking internship opportunities starting in September 2024 for 3 to 6 months. I am highly committed and
self-driven individual to continuously improving skills relevant to the fast-paced technology industry.

## 2 semester programme


Bachelor of Computer Science with Honours
Universiti Teknologi MARA Kampus Tapah

Developed a web app that integrates custom trained model to recognize major guitar chord using visual data.
Utilized Tensorflow and Keras framewo

In [ ]:
#Save the extracted text into .md format file



### Extract from scanned PDFs/images

In [18]:
# Reading and extracting texts from scanned PDFs
# By using easyOCR (en,ms)
import easyocr
from pdf2image import convert_from_path
reader = easyocr.Reader(['en','ms'])


In [19]:

poppler_path = r"C:/Users/User/poppler-24.08.0/Library/bin"
pdf_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/personal_documents/spm_certificate.pdf"
import numpy as np
images = convert_from_path(pdf_path,poppler_path=poppler_path)

all_text = []
# Process each page image directly
for i, img in enumerate(images):
    # Convert PIL image to NumPy array
    img_np = np.array(img)

    #read the text from image
    text = reader.readtext(img_np, detail=0) #extracts the text 
    page_text = "\n".join(text) #joins the text lines
    all_text.append(f"## Page{i+1}\n{page_text}") #format each page by including a title/number of pages

markdown_text = "\n\n".join(all_text)
print(markdown_text)
    


## Page1
KEMENTERIAN PENDIDIKAN MALAYSIA
MINISTRY OF EDUCATION MALAYSIA
LEMBAGA PEPERIKSAAN
EXAMINATIONS SYNDICATE
Calon yang namanya tercatat di bawah telah menduduki peperiksaan Sijil Pelajaran Malaysia (SPM) dan
layak dianugerahi
SIJIL PELAJARAN MALAYSIA
MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR
020630-02-0849
KC208A089
SMK AGAMA KEDAH
Mata Pelajaran
Gred
Subject
Grade
BAHASA MELAYU
A
(CEMERLANG TINGGI)
BAHASA INGGERIS
A
(CEMERLANG TINGGI)
PENDIDIKAN ISLAM
A+
(CEMERLANG TERTINGGI)
SEJARAH
A+
(CEMERLANG TERTINGGI)
MATHEMATICS
A
(CEMERLANG TINGGI)
BAHASA ARAB
B+ (KEPUJIAN TERTINGGI)
ADDITIONAL MATHEMATICS
C
(KEPUJIAN)
PHYSICS
A-
(CEMERLANG)
CHEMISTRY
A
(CEMERLANG TINGGI)
BIOLOGY
A+
(CEMERLANG TERTINGGI)
HIFZ AL-QURAN
B+ (KEPUJIAN TERTINGGI)
MAHARAT AL-QURAN
B+ (KEPUJIAN TERTINGGI)
JUMLAH MATA PELAJARAN DUA BELAS
PEPERIKSAAN TAHUN 2019
Pengarah Peperiksaan
Director of Examinations
191258556
Kementerian Pendidikan Malaysia
A
04845697
Ministry of Education Malaysia
2423
iaur
SebTaheh MuTu

In [ ]:
# Save the extracted text into .md format

In [ ]:
# Reading and extracting texts from .docx/.docs



## STEP 2: Chunking the extracted text

### Implement hybrid chunking (Split based on the headings of the markdown + fixed-length split )


In [26]:
# use the md_text as an example

from markdown_it import MarkdownIt

md = MarkdownIt() #initialize the library
print(md.render(md_text))

<h1>t s ters</h1>
<p>Current CGPA 3.49</p>
<h2>PROFESSIONAL SUMMARY</h2>
<p>A final year student of Bachelor in Computer Science with the goal to seek and expand knowledge and
specialize in Software Engineering field. With a strong passion for software engineering, I am actively
seeking internship opportunities starting in September 2024 for 3 to 6 months. I am highly committed and
self-driven individual to continuously improving skills relevant to the fast-paced technology industry.</p>
<h2>2 semester programme</h2>
<p>Bachelor of Computer Science with Honours
Universiti Teknologi MARA Kampus Tapah</p>
<p>Developed a web app that integrates custom trained model to recognize major guitar chord using visual data.
Utilized Tensorflow and Keras framework for model development</p>
<p>Module 3 Science Stream
Kedah Matriculation College</p>
<p>Collaborated and developed machine learning models to suggest optimal conditions for some aspects of farming.
Utilized SVM, KNN, Decision Tree, and St

In [27]:
from collections import defaultdict
from pathlib import Path

def split_markdown_by_headings(md_text):
    md = MarkdownIt()  # Initialize Markdown parser
    tokens = md.parse(md_text)  # Parse Markdown
    sections = defaultdict(list)  # Store content based on headings / split content based on headings
    
    current_heading = "No Heading"  # Default if no heading at start

    for token in tokens:
        if token.type == "heading_open":  # Detect heading (h1, h2, h3, etc.)
            level = int(token.tag[1])  # Extract heading level (1, 2, 3)
            next_token = tokens[tokens.index(token) + 1]  # Get heading text
            if next_token.type == "inline":
                current_heading = f"{'#' * level} {next_token.content}"  # Format heading
        
        elif token.type == "paragraph_open":  # Detect paragraphs
            next_token = tokens[tokens.index(token) + 1]
            if next_token.type == "inline":
                sections[current_heading].append(next_token.content)  # Store content under heading

    # Convert dictionary values to strings
    return {key: "\n".join(value) for key, value in sections.items()}


split_sections = split_markdown_by_headings(md_text)
print(split_sections)



{'# t s ters': 'Current CGPA 3.49', '## PROFESSIONAL SUMMARY': 'A final year student of Bachelor in Computer Science with the goal to seek and expand knowledge and\nspecialize in Software Engineering field. With a strong passion for software engineering, I am actively\nseeking internship opportunities starting in September 2024 for 3 to 6 months. I am highly committed and\nself-driven individual to continuously improving skills relevant to the fast-paced technology industry.', '## 2 semester programme': 'Bachelor of Computer Science with Honours\nUniversiti Teknologi MARA Kampus Tapah\nDeveloped a web app that integrates custom trained model to recognize major guitar chord using visual data.\nUtilized Tensorflow and Keras framework for model development\nModule 3 Science Stream\nKedah Matriculation College\nCollaborated and developed machine learning models to suggest optimal conditions for some aspects of farming.\nUtilized SVM, KNN, Decision Tree, and Streamlit for this assignment pr

In [28]:
import nltk

# for each sections, check if the tokens in a section is more than 512 tokens, divide the section into more subsetions by using NLTK

# function to chunk the text if it has more than 512 tokens
def chunk_text(text, max_tokens=512):
    words = nltk.word_tokenize(text)
    chunks = []

    for i in range(0, len(words), max_tokens):
        chunk_words = words[i: i + max_tokens]
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
    return chunks
    
new_sections = {}


for heading, content in split_sections.items():
    tokens = nltk.word_tokenize(content)  # Count tokens in the content
    
    if len(tokens) > 512:
        print(f"Chunking '{heading}' as it has {len(tokens)} tokens...")
        chunked_subsections = chunk_text(content)  # Split content into 512-token chunks
        
        # Store each chunk under new subsections
        for i, chunk in enumerate(chunked_subsections, start=1):
            new_heading = f"{heading} (Part {i})"  # Create subheading
            new_sections[new_heading] = chunk
    else:
        new_sections[heading] = content  # Keep original section
    

    print(new_sections.items())


dict_items([('# t s ters', 'Current CGPA 3.49')])
dict_items([('# t s ters', 'Current CGPA 3.49'), ('## PROFESSIONAL SUMMARY', 'A final year student of Bachelor in Computer Science with the goal to seek and expand knowledge and\nspecialize in Software Engineering field. With a strong passion for software engineering, I am actively\nseeking internship opportunities starting in September 2024 for 3 to 6 months. I am highly committed and\nself-driven individual to continuously improving skills relevant to the fast-paced technology industry.')])
dict_items([('# t s ters', 'Current CGPA 3.49'), ('## PROFESSIONAL SUMMARY', 'A final year student of Bachelor in Computer Science with the goal to seek and expand knowledge and\nspecialize in Software Engineering field. With a strong passion for software engineering, I am actively\nseeking internship opportunities starting in September 2024 for 3 to 6 months. I am highly committed and\nself-driven individual to continuously improving skills releva

## Step 3: Save the chunked data into a json format file

In [30]:
import json
import nltk
from nltk.tokenize import word_tokenize
from pathlib import Path

chunk_title = "Resume"
source = "current_resume"
# Path to the existing JSON file
output_path = "processed_data.json"

# Load existing data if file exists, otherwise start with an empty list
if Path(output_path).exists():
    with open(output_path, "r", encoding="utf-8") as file:
        try:
            existing_data = json.load(file)
            if not isinstance(existing_data, list):  # Ensure it's a list
                existing_data = []
        except json.JSONDecodeError:  # Handle empty or corrupted JSON
            existing_data = []
else:
    existing_data = []

# Convert new chunks to JSON format with token count
new_json_data = []
for heading, content in new_sections.items():
    token_count = len(word_tokenize(content))  # Count tokens
    new_json_data.append({
        "chunk_id": chunk_title,
        "heading": heading,
        "content": content,
        "source": source,
        "tokens": token_count
    })
    chunk_counter=chunk_counter+1

# Append new data to existing data
existing_data.extend(new_json_data)

# Save the updated JSON back to the file
with open(output_path, "w", encoding="utf-8") as file:
    json.dump(existing_data, file, indent=4, ensure_ascii=False)

print(f"New document chunks appended to {output_path}")


New document chunks appended to processed_data.json


# Pipeline to read pdf files in a folder and extract it's text

In [17]:
import os
import pymupdf
import pymupdf4llm
import easyocr
from pdf2image import convert_from_path
from markdown_it import MarkdownIt
from collections import defaultdict
from pathlib import Path
import nltk
from nltk.tokenize import word_tokenize
import json

In [9]:
# Chunking function definition (Part 1: Chunk based on the headings) 
def split_markdown_by_headings(md_text):
    md = MarkdownIt()  # Initialize Markdown parser
    tokens = md.parse(md_text)  # Parse Markdown
    sections = defaultdict(list)  # Store content based on headings / split content based on headings
    
    current_heading = "No Heading"  # Default if no heading at start

    for token in tokens:
        if token.type == "heading_open":  # Detect heading (h1, h2, h3, etc.)
            level = int(token.tag[1])  # Extract heading level (1, 2, 3)
            next_token = tokens[tokens.index(token) + 1]  # Get heading text
            if next_token.type == "inline":
                current_heading = f"{'#' * level} {next_token.content}"  # Format heading
        
        elif token.type == "paragraph_open":  # Detect paragraphs
            next_token = tokens[tokens.index(token) + 1]
            if next_token.type == "inline":
                sections[current_heading].append(next_token.content)  # Store content under heading

    # Convert dictionary values to strings
    return {key: "\n".join(value) for key, value in sections.items()}

In [10]:
# Part 2: function to chunk the text if it has more than 512 tokens 
def chunk_text(text, max_tokens=512):
    words = nltk.word_tokenize(text)
    chunks = []

    for i in range(0, len(words), max_tokens):
        chunk_words = words[i: i + max_tokens]
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
    return chunks
    

In [19]:


#Specify the folder 
folder_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/agreements"
# Path to the existing JSON file
output_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/scripts/processed_data.json"


#Loop through all PDFs in the folder specified by folder_path
for file in os.listdir(folder_path):
    if file.endswith(".pdf"): #if the file has .pdf extension
        pdf_path = os.path.join(folder_path, file) # create the full path for each pdf files

        doc = pymupdf4llm.to_markdown(pdf_path) #extract the texts in the pdf into .md format

        # Part 1 Chunking (Heading-based)
        split_doc_sections = split_markdown_by_headings(doc)

        # Part 2 Chunking (split further if tokens>512)

        new_doc_sections = {} #new dict that stores much more splitted chunks

        for heading, content in split_doc_sections.items():
            tokens = nltk.word_tokenize(content)  # Count tokens in the content
            
            if len(tokens) > 512:
                chunked_subsections = chunk_text(content)  # Split content into 512-token chunks
                
                # Store each chunk under new subsections
                for i, chunk in enumerate(chunked_subsections, start=1):
                    new_heading = f"{heading} (Part {i})"  # Create subheading
                    new_doc_sections[new_heading] = chunk
            else:
                new_doc_sections[heading] = content  # Keep original section

        # Append the new data chunks to the .json file
        
        chunk_title = os.path.splitext(os.path.basename(pdf_path))[0]
        
        source = os.path.splitext(os.path.basename(pdf_path))[0]


        # Load existing data if file exists, otherwise start with an empty list
        if Path(output_path).exists():
            with open(output_path, "r", encoding="utf-8") as file:
                try:
                    existing_data = json.load(file)
                    if not isinstance(existing_data, list):  # Ensure it's a list
                        existing_data = []
                except json.JSONDecodeError:  # Handle empty or corrupted JSON
                    existing_data = []
        else:
            existing_data = []

        # Convert new chunks to JSON format with token count
        new_json_data = []
        for heading, content in new_doc_sections.items():
            token_count = len(word_tokenize(content))  # Count tokens
            new_json_data.append({
                "chunk_id": chunk_title,
                "heading": heading,
                "content": content,
                "source": source,
                "tokens": token_count
            })


        # Append new data to existing data
        existing_data.extend(new_json_data)

        # Save the updated JSON back to the file
        with open(output_path, "w", encoding="utf-8") as file:
            json.dump(existing_data, file, indent=4, ensure_ascii=False)

        print(f"New document chunks appended to {output_path}")




Processing C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/raw_documents/agreements\ASB-ASB2-Application-Form-ENG-BM.pdf...
[                                        ] (0/1=[==                                      ] ( 1/19=[====                                    ] ( 2/19=[======                                  ] ( 3/19=[========                                ] ( 4/19=[==========                              ] ( 5/19=[============                            ] ( 6/19=[==============                          ] ( 7/19=[================                        ] ( 8/19=[==================                      ] ( 9/19==[=====================                   ] (10/1=[=======================                 ] (11/1=[=========================               ] (12/1=[===========================             ] (13/1=[=============================           ] (14/1=[===============================         ] (15/1=[=================================       ] (16/1=[=====

### *The Agreements, KTM train ticket, and Resume are extracted* ###

Just go on with these documents first for RAG 

In [21]:
import uuid
import json

# Read data from processed_data.json
with open("C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/scripts/processed_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Replace chunk_id with a unique UUID
for entry in data:
    entry["chunk_id"] = str(uuid.uuid4())

# Save the updated data back to the file
with open("processed_data.json", "w") as file:
    json.dump(data, file, indent=4)

# Print updated data
print(json.dumps(data, indent=4))

[
    {
        "chunk_id": "dc42bf3a-c3ba-4232-abb1-8d25de8f0039",
        "heading": "### 16/11/2023 12:46 PM 16/11/2023 03:57 PM",
        "content": "MUHAMMAD ALIF SAFWAN BIN SYAMSYUL SYAHAR - Adult\nxxxxxxxx0849",
        "source": "train_ticket_receipt",
        "tokens": 9
    },
    {
        "chunk_id": "1b90c6c7-e5af-49b9-a2fa-328f2b723c5f",
        "heading": "#### MYR 46.00/DebitCard",
        "content": "K231189695108 ** T231175999163 (Single)",
        "source": "train_ticket_receipt",
        "tokens": 7
    },
    {
        "chunk_id": "01b171ea-76a7-49d5-800e-5affc2cbb65d",
        "heading": "#### Online",
        "content": "Airasia Ride Promo Code : KTMRIDE\nSila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.\nTertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.\nKTMB CALL CENTER +603-2267 1200",
        "source": "train_ticket_receipt",
        "tokens": 43
    },
    {